# Installs and Imports

In [ ]:
!pip install pandas transformers datasets scikit-learn torch transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.model_selection import KFold
from tqdm import tqdm
from transformers import EarlyStoppingCallback
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Drive Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Loading

In [ ]:
annotated_and_pseudolabeled_path = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Dataset/ANNOTATED_AND_PSEUDOLABELED_DATA_01.xlsx'
cleaned_preprocessed_path = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Dataset/CLEANED_PREPROCESSED_DATA_05.xlsx'

df_combined_nd = pd.read_excel(annotated_and_pseudolabeled_path, sheet_name='ND')
df_combined_dn = pd.read_excel(annotated_and_pseudolabeled_path, sheet_name='DN')

In [ ]:
model_path_nd = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/ND'
model_path_dn = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/DN'

In [ ]:
tokenizer_xlm = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

# Final Training and Evaluation

In [ ]:
def preprocess_data(df, tokenizer, max_length=512):
    texts = df['text'].tolist()
    labels = df['numericalLabel'].astype(int).tolist()
    inputs = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    inputs['labels'] = torch.tensor(labels)
    return inputs

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
inputs_nd = preprocess_data(df_combined_nd, tokenizer_xlm)
inputs_dn = preprocess_data(df_combined_dn, tokenizer_xlm)

In [ ]:
dataset_nd = CustomDataset(inputs_nd)
dataset_dn = CustomDataset(inputs_dn)

In [ ]:
def calculate_class_weights(labels):
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
    return {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:
class_weights_nd = calculate_class_weights(df_combined_nd['numericalLabel'].astype(int))
class_weights_dn = calculate_class_weights(df_combined_dn['numericalLabel'].astype(int))

In [ ]:
def create_model_with_class_weights(model_path, num_labels, class_weights):
    config = XLMRobertaConfig.from_pretrained(model_path, num_labels=num_labels)
    model = XLMRobertaForSequenceClassification.from_pretrained(model_path, config=config)
    model.config.class_weights = class_weights
    return model

In [ ]:
def forward_with_class_weights(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
    outputs = self.roberta(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        head_mask=head_mask,
        inputs_embeds=inputs_embeds,
    )

    sequence_output = outputs[0]
    logits = self.classifier(sequence_output)

    loss = None
    if labels is not None:
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor([self.config.class_weights[i] for i in range(self.config.num_labels)]).float().to(self.device))
        loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

    return SequenceClassifierOutput(
        loss=loss,
        logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

In [ ]:
XLMRobertaForSequenceClassification.forward = forward_with_class_weights

In [ ]:
def train_with_best_params(best_params, category_name, model_path, train_dataset, class_weights):
    training_args = TrainingArguments(
        output_dir=f'/tmp/{category_name}',  # Temporary location to avoid saving to Google Drive
        evaluation_strategy='epoch',
        save_strategy='no',  # Don't save every model
        learning_rate=best_params['learning_rate'],
        per_device_train_batch_size=best_params['per_device_train_batch_size'],
        per_device_eval_batch_size=16,
        num_train_epochs=best_params['num_train_epochs'],
        weight_decay=best_params['weight_decay'],
        warmup_steps=best_params['warmup_steps'],
        gradient_accumulation_steps=best_params['gradient_accumulation_steps'],
        load_best_model_at_end=False,
        metric_for_best_model='eval_loss',
        greater_is_better=False,
        fp16=True,  # Use mixed precision training
    )

    # Initialize K-fold cross-validator
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Store evaluation results
    eval_results_list = []

    # Perform K-Fold cross-validation and train final model
    for fold, (train_index, eval_index) in enumerate(kf.split(train_dataset)):
        print(f"Starting fold {fold+1}...")
        train_subset = torch.utils.data.Subset(train_dataset, train_index)
        eval_subset = torch.utils.data.Subset(train_dataset, eval_index)

        model = create_model_with_class_weights(model_path, num_labels=7, class_weights=class_weights)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer_xlm)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_subset,
            eval_dataset=eval_subset,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )

        for epoch in range(training_args.num_train_epochs):
            start_time = time.time()
            print(f"Training epoch {epoch+1}/{training_args.num_train_epochs}")
            trainer.train()
            end_time = time.time()
            epoch_time = end_time - start_time
            print(f"Time taken for epoch {epoch+1}: {epoch_time:.2f} seconds")
            # Clear cache
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

        # Evaluate the model
        eval_results = trainer.evaluate(eval_subset)
        print(f"Evaluation results for fold {fold+1}: {eval_results}")
        eval_results_list.append(eval_results)

    # Print final evaluation results
    print("Final evaluation results:")
    for i, results in enumerate(eval_results_list):
        print(f"Fold {i+1}: {results}")

In [ ]:
best_params_nd = {
    "learning_rate": 3e-5,
    "num_train_epochs": 3,
    "per_device_train_batch_size": 16,
    "gradient_accumulation_steps": 2,
    "weight_decay": 0.01,
    "warmup_steps": 100
}

best_params_dn = {
    "learning_rate": 2e-5,
    "num_train_epochs": 4,
    "per_device_train_batch_size": 32,
    "gradient_accumulation_steps": 2,
    "weight_decay": 0.1,
    "warmup_steps": 200
}

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput

In [ ]:
import time

In [ ]:
train_with_best_params(best_params_nd, 'ND', model_path_nd, dataset_nd, class_weights_nd)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting fold 1...
Training epoch 1/3


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.092584,0.971882,0.972078,0.972917,0.971882
2,0.091300,0.051894,0.981975,0.981996,0.982072,0.981975


Time taken for epoch 1: 523.46 seconds
Training epoch 2/3


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.092898,0.971882,0.971789,0.971880,0.971882
2,0.066200,0.076098,0.977650,0.977645,0.977687,0.977650


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.095235,0.977633,0.977723,0.978378,0.977633
2,0.070300,0.029653,0.987013,0.987059,0.987189,0.987013


Time taken for epoch 2: 524.43 seconds
Training epoch 3/3


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.071695,0.978355,0.978344,0.978404,0.978355
2,0.055800,0.051088,0.984848,0.984910,0.985140,0.984848


Time taken for epoch 3: 524.47 seconds


Evaluation results for fold 5: {'eval_loss': 0.05108775198459625, 'eval_accuracy': 0.9848484848484849, 'eval_f1': 0.9849095640846939, 'eval_precision': 0.985139838865886, 'eval_recall': 0.9848484848484849, 'eval_runtime': 11.0878, 'eval_samples_per_second': 125.002, 'eval_steps_per_second': 7.846, 'epoch': 2.9913544668587897}
Final evaluation results:
Fold 1: {'eval_loss': 0.0881132036447525, 'eval_accuracy': 0.9726027397260274, 'eval_f1': 0.972595741436196, 'eval_precision': 0.9726870506163549, 'eval_recall': 0.9726027397260274, 'eval_runtime': 11.1219, 'eval_samples_per_second': 124.709, 'eval_steps_per_second': 7.822, 'epoch': 2.9913544668587897}
Fold 2: {'eval_loss': 0.09422948211431503, 'eval_accuracy': 0.9711607786589762, 'eval_f1': 0.9715642842615533, 'eval_precision': 0.9730475293424713, 'eval_recall': 0.9711607786589762, 'eval_runtime': 11.1393, 'eval_samples_per_second': 124.514, 'eval_steps_per_second': 7.81, 'epoch': 2.9913544668587897}
Fold 3: {'eval_loss': 0.1298840939998

In [ ]:
train_with_best_params(best_params_dn, 'DN', model_path_dn, dataset_dn, class_weights_dn)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting fold 1...
Training epoch 1/4


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.188242,0.955357,0.955880,0.958294,0.955357
2,No log,0.204450,0.939732,0.941504,0.947752,0.939732
3,No log,0.187160,0.944196,0.945470,0.950430,0.944196
4,No log,0.196807,0.946429,0.947818,0.952791,0.946429


Time taken for epoch 1: 213.46 seconds
Training epoch 2/4


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.183093,0.948661,0.949784,0.954018,0.948661
2,No log,0.182375,0.944196,0.945870,0.951629,0.944196
3,No log,0.195221,0.941964,0.943611,0.949813,0.941964
4,No log,0.217428,0.941964,0.943367,0.948886,0.941964


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.200694,0.955357,0.956091,0.958278,0.955357
2,No log,0.202573,0.962054,0.962311,0.963355,0.962054
3,No log,0.215840,0.953125,0.953516,0.954947,0.953125
4,No log,0.220644,0.950893,0.951328,0.952872,0.950893


Time taken for epoch 3: 214.54 seconds
Training epoch 4/4


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.231614,0.953125,0.953474,0.954798,0.953125
2,No log,0.237105,0.953125,0.953573,0.955080,0.953125
3,No log,0.283005,0.946429,0.946521,0.947183,0.946429
4,No log,0.278509,0.937500,0.937949,0.939663,0.937500


Time taken for epoch 4: 214.44 seconds


Evaluation results for fold 5: {'eval_loss': 0.27850863337516785, 'eval_accuracy': 0.9375, 'eval_f1': 0.9379487732521646, 'eval_precision': 0.9396627653192836, 'eval_recall': 0.9375, 'eval_runtime': 3.6344, 'eval_samples_per_second': 123.267, 'eval_steps_per_second': 7.704, 'epoch': 4.0}
Final evaluation results:
Fold 1: {'eval_loss': 0.23280498385429382, 'eval_accuracy': 0.9441964285714286, 'eval_f1': 0.9453816425231478, 'eval_precision': 0.9496678016468939, 'eval_recall': 0.9441964285714286, 'eval_runtime': 3.629, 'eval_samples_per_second': 123.451, 'eval_steps_per_second': 7.716, 'epoch': 4.0}
Fold 2: {'eval_loss': 0.3177950382232666, 'eval_accuracy': 0.9352678571428571, 'eval_f1': 0.9355148252669375, 'eval_precision': 0.9368469443698532, 'eval_recall': 0.9352678571428571, 'eval_runtime': 3.6247, 'eval_samples_per_second': 123.596, 'eval_steps_per_second': 7.725, 'epoch': 4.0}
Fold 3: {'eval_loss': 0.15998150408267975, 'eval_accuracy': 0.9464285714285714, 'eval_f1': 0.94893272148743

In [ ]:
def create_model_with_class_weights_for_evaluation(model_path, num_labels, class_weights):
    config = XLMRobertaConfig.from_pretrained(model_path, num_labels=num_labels)
    model = XLMRobertaForSequenceClassification.from_pretrained(model_path, config=config)
    model.config.class_weights = class_weights
    return model

In [ ]:
def evaluate_model(model_path, test_dataset, class_weights):
    model = create_model_with_class_weights_for_evaluation(model_path, num_labels=7, class_weights=class_weights)
    trainer = Trainer(
        model=model,
        compute_metrics=compute_metrics
    )
    eval_results = trainer.evaluate(test_dataset)
    print(eval_results)

In [ ]:
evaluate_model(model_path_nd, dataset_nd, class_weights_nd)

{'eval_loss': 0.04604846611618996, 'eval_accuracy': 0.9877397951824607, 'eval_f1': 0.9877760266516967, 'eval_precision': 0.9878980369920097, 'eval_recall': 0.9877397951824607, 'eval_runtime': 54.5471, 'eval_samples_per_second': 127.101, 'eval_steps_per_second': 15.895}


In [ ]:
evaluate_model(model_path_dn, dataset_dn, class_weights_dn)

{'eval_loss': 0.2489369511604309, 'eval_accuracy': 0.9517857142857142, 'eval_f1': 0.9518624860528283, 'eval_precision': 0.9522500275594248, 'eval_recall': 0.9517857142857142, 'eval_runtime': 17.6529, 'eval_samples_per_second': 126.892, 'eval_steps_per_second': 15.861}


In [ ]:
def save_model(model, tokenizer, category_name):
    model.save_pretrained(f'/content/{category_name}_final_model')
    tokenizer.save_pretrained(f'/content/{category_name}_final_model')
    print(f"Model saved to /content/{category_name}_final_model")

In [41]:
def save_model_to_drive(model, tokenizer, drive_path):
    model.save_pretrained(drive_path)
    tokenizer.save_pretrained(drive_path)
    print(f"Model saved to {drive_path}")

In [42]:
model_nd = create_model_with_class_weights_for_evaluation(model_path_nd, num_labels=7, class_weights=class_weights_nd)
save_model_to_drive(model_nd, tokenizer_xlm, '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/ND_final')

Model saved to /content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/ND_final


In [43]:
model_dn = create_model_with_class_weights_for_evaluation(model_path_dn, num_labels=7, class_weights=class_weights_dn)
save_model_to_drive(model_dn, tokenizer_xlm, '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/DN_final')

Model saved to /content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/DN_final
